In [2]:
#Importing the Libraries
import pandas as pd

In [3]:
#Loading Data
fighter_stats = pd.read_excel('./fighter_stats.xlsx')
fighter_stats.head()
fighter_stats.rename(columns={'Ht.': 'height', 'Wt.': 'weight'}, inplace=True)

# Analysing Dataset

In [4]:
fighter_stats.head()

,First,Last,Nickname,height,weight,Reach,Stance,W,L,D,Belt
0,Tom,Aaron,NaN,--,155 lbs.,--,NaN,5,3,0,NaN
1,Danny,Abbadi,The Assassin,"5' 11""",155 lbs.,--,Orthodox,4,6,0,NaN
2,David,Abbott,Tank,"6' 0""",265 lbs.,--,Switch,10,14,0,NaN
3,Shamil,Abdurakhimov,Abrek,"6' 3""",235 lbs.,"76.0""",Orthodox,18,4,0,NaN
4,Hiroyuki,Abe,Abe Ani,"5' 6""",145 lbs.,--,Orthodox,8,14,3,NaN


# Feature Creation

## We are comparing the players based on follwing 4 attributes:
###    1) Win Percentage 
            - Given the heightest weightage
###    2) Height
###    3) Stance
            - Changed into categorical variable and gave integral value to each of the stance
###    4) Reach

In [10]:
#Feature 1 - Win Percentage = Win/TotalFights
fighter_stats['total_fights'] = fighter_stats.W + fighter_stats.L + fighter_stats.D
fighter_stats['win_percent'] = fighter_stats.W/(fighter_stats.total_fights)

In [14]:
#Feature - 2
#Unique Values of the stance
#print(nhl_stat_fighter.Stance.value_counts())

#Assigning weights to the stances
#Orthodox - 1
#Southpaw - 2
#Switch - 3
#Open Stance - 4
#Sideways - 5

Orthodox_int = 1
Southpaw_int = 2
Switch_int = 3
Open_Stance_int = 4
Sideways_int = 5

fighter_stats['stance_int'] = 0
fighter_stats.loc[fighter_stats.Stance == 'Orthodox','stance_int'] =Orthodox_int
fighter_stats.loc[fighter_stats.Stance == 'Southpaw','stance_int'] =Southpaw_int
fighter_stats.loc[fighter_stats.Stance == 'Switch','stance_int'] =Switch_int
fighter_stats.loc[fighter_stats.Stance == 'Open Stance','stance_int'] =Open_Stance_int
fighter_stats.loc[fighter_stats.Stance == 'Sideways','stance_int'] =Sideways_int
fighter_stats.loc[:,('stance_int','Stance')].head()

,stance_int,Stance
0,0,NaN
1,1,Orthodox
2,3,Switch
3,1,Orthodox
4,1,Orthodox


In [16]:
import re
#Feature - 3: Making height to integer
#1 inch = 0.0833333ft

fighter_stats['height_int'] = 0

#Processing feet part of height
fighter_stats['height_ft'] =fighter_stats.height.str.split("'").str[0]
fighter_stats.height_ft.replace('--',0,inplace = True)
fighter_stats['height_ft'] = pd.to_numeric(fighter_stats['height_ft'])

#Processing inch part of the hight
fighter_stats['height_inch'] = fighter_stats.height.str.split("'").str[1].str.extract('([0-9]+)[\"]?', expand=False)
fighter_stats['height_inch'] = pd.to_numeric(fighter_stats['height_inch'], downcast='integer')

#Creating final height
fighter_stats['height_int'] = fighter_stats['height_ft'] +  0.0833333 * fighter_stats['height_inch']
fighter_stats.height_int.fillna(0,inplace=True)

In [17]:
#Feature - 4: Making reach integer
fighter_stats.Reach.value_counts()
fighter_stats.Reach.replace('--',0,inplace = True)
fighter_stats['reach_int'] = 0
fighter_stats.reach_int = fighter_stats.Reach.str.split(".").str[0]

fighter_stats.reach_int.fillna(0,inplace=True)
fighter_stats.reach_int = pd.to_numeric(fighter_stats.reach_int, downcast='integer')

In [19]:
#Giving weightage to Attributes
ht_weightage = .1
reach_weightage = .1
stance_weightage = .1
pastwin_percent_weightage = .7

In [20]:
#Setting the player ID - Fetch from figths data, currently hardcoded
first_player = 0
second_player = 2

#Getting the players attributes
first_df = fighter_stats[fighter_stats.index == first_player] 
second_df = fighter_stats[fighter_stats.index == second_player]

#Scoring Players:

first_player_score = 0
second_player_score = 0

#Compare if both the values are available

#If both heights are available
if first_df.height_int.iloc[0] > 0 and second_df.height_int.iloc[0] > 0:
    print('here1')
    first_player_score = first_df.height_int.iloc[0] * ht_weightage
    second_player_score = second_df.height_int.iloc[0] * ht_weightage
    
#If both reach are available
if first_df.reach_int.iloc[0] > 0 and second_df.reach_int.iloc[0] > 0:
    print('here2')
    first_player_score = first_player_score + first_df.reach_int.iloc[0] * reach_weightage
    second_player_score = second_player_score +  second_df.reach_int.iloc[0] * reach_weightage
    

#If both stance are available
if first_df.stance_int.iloc[0]  > 0 and second_df.stance_int.iloc[0] > 0:
    print('here3')
    first_player_score = first_player_score + first_df.stance_int.iloc[0] * stance_weightage
    second_player_score = second_player_score +  second_df.stance_int.iloc[0] * stance_weightage

#If both the win percent is given
if first_df.win_percent.iloc[0]  > 0 and second_df.win_percent.iloc[0] > 0:
    first_player_score = first_player_score + first_df.win_percent.iloc[0] * 10 * pastwin_percent_weightage
    second_player_score = second_player_score +  second_df.win_percent.iloc[0] * 10 * pastwin_percent_weightage

#Print out the scores
print('First player Score: ' + str(first_player_score))
print('Second player Score: ' + str(second_player_score))

#Comparing the score
if first_player_score > second_player_score:
    print('First Player is the Winner')
elif first_player_score < second_player_score:
    print('Second Player is the Winner')
else:
    print('Need More data to Predict')

First player Score: 4.375
Second player Score: 2.91666666667
First Player is the Winner


## Top 5 fighters per weight class ranked by win percentage

In [21]:
MINIMUM_FIGHTS_TO_HAVE = 10
for weight_class in fighter_stats.weight.unique():
    fighters_in_weight_class = fighter_stats[fighter_stats.weight == weight_class]
    fighters_in_weight_class = fighters_in_weight_class[(fighters_in_weight_class.W +\
                                                        fighters_in_weight_class.L + \
                                                        fighters_in_weight_class.D )>= MINIMUM_FIGHTS_TO_HAVE]
    # Rank them by win percentage
    fighters_in_weight_class = fighters_in_weight_class.sort_values(by=['win_percent', 'total_fights'],ascending=[False, False])
    top_5_fighters = fighters_in_weight_class.loc[:, 
        ['First', 'Last', 'total_fights', 'weight','win_percent']].head()
    print(top_5_fighters.to_csv(index=False))
    print('==================================================================================================================')

First,Last,total_fights,weight,win_percent
Khabib,Nurmagomedov,25,155 lbs.,1.0
Gregor,Gillespie,11,155 lbs.,1.0
Justin,Gaethje,19,155 lbs.,0.9473684210526315
Islam,Makhachev,16,155 lbs.,0.9375
Alan,Patrick,16,155 lbs.,0.9375

First,Last,total_fights,weight,win_percent
Curtis,Blaydes,10,265 lbs.,0.9
Sean,McCorkle,15,265 lbs.,0.8666666666666667
Shane,Carwin,14,265 lbs.,0.8571428571428571
Justin,Wren,13,265 lbs.,0.8461538461538461
Jared,Rosholt,17,265 lbs.,0.8235294117647058

First,Last,total_fights,weight,win_percent
Mike,Ciesnolevicz,23,235 lbs.,0.8260869565217391
Shamil,Abdurakhimov,22,235 lbs.,0.8181818181818182
Dion,Staring,32,235 lbs.,0.75
Josh,Hendricks,25,235 lbs.,0.72
Justin,Eilers,27,235 lbs.,0.7037037037037037

First,Last,total_fights,weight,win_percent
Brian,Ortega,13,145 lbs.,1.0
Julio Cesar,Neves Jr.,34,145 lbs.,0.9705882352941176
Cris,Cyborg,20,145 lbs.,0.95
Alexander,Volkanovski,18,145 lbs.,0.9444444444444444
Zabit,Magomedsharipov,15,145 lbs.,0.9333333333333333

First,Last

First,Last,total_fights,weight,win_percent
Oleg,Taktarov,24,210 lbs.,0.7083333333333334
Enson,Inoue,20,210 lbs.,0.6
Marco,Ruas,15,210 lbs.,0.6
Joe,Pardo,10,210 lbs.,0.4

First,Last,total_fights,weight,win_percent
Renan,Barao,41,140 lbs.,0.8780487804878049

First,Last,total_fights,weight,win_percent
Eric,Bedard,13,239 lbs.,0.46153846153846156

First,Last,total_fights,weight,win_percent
Eun Soo,Lee,18,215 lbs.,0.8333333333333334
Achmed,Labasanov,13,215 lbs.,0.6153846153846154
Rodney,Faverus,47,215 lbs.,0.5319148936170213

First,Last,total_fights,weight,win_percent
Megumi,Fujii,29,113 lbs.,0.896551724137931

First,Last,total_fights,weight,win_percent
Josh,Bennett,25,259 lbs.,0.48

First,Last,total_fights,weight,win_percent
Andre,Roberts,17,345 lbs.,0.8235294117647058
Dan,Bobish,26,345 lbs.,0.6538461538461539

First,Last,total_fights,weight,win_percent
Derrick,Lewis,23,260 lbs.,0.782608695652174
Todd,Duffee,12,260 lbs.,0.75
Tim,Hague,18,260 lbs.,0.7222222222222222
Shawn,Jordan,25,260 lbs.,